## Objectives

* Prepare sequence data to use in a recurrent neural network (RNN)
* Build and train a model to perform word prediction

In [ ]:
import os 
import pandas as pd

nyt_dir = 'data/nyt_dataset/articles/'

all_headlines = []
for filename in os.listdir(nyt_dir):
    if 'Articles' in filename:
        # Read in all the data from the CSV file
        headlines_df = pd.read_csv(nyt_dir + filename)
        # Add all of the headlines to our list
        all_headlines.extend(list(headlines_df.headline.values))
len(all_headlines)

In [ ]:
all_headlines[:20]

## Cleaning the Data

In [ ]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

In [ ]:
all_headlines[:20]

## Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

In [ ]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

In [ ]:
# To see how the tokenizer actually saves the words
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

## Creating Sequences

In [ ]:
# Convert data to sequence of tokens 
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

## Padding Sequences

In [ ]:
# To make all the sequences the same length

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

## Creating Predictors and Target

In [ ]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

Like our earlier sections, these targets are categorical. We are predicting one word out of our possible total vocabulary. Instead of the network predicting scalar numbers, we will have it predict binary categories. 

In [ ]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

## Creating the Model

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1 

model = Sequential()

# Add input embedding layer
# Layer takes the tokenized sequence and learns an embedding for all the words in the dataset.
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
# Type of RNN which, unlike feed forward neural networks, has loops in them
# Normal RNNs suffer from information recency bias so LSTMs provide a way to retain longer term information.
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.summary()

## Compiling the Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Training the Model

In [ ]:
model.fit(predictors, labels, epochs=30, verbose=1)

## Making Predictions

In [ ]:
def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    prediction = model.predict_classes(token_list, verbose=0)
    return prediction

In [ ]:
prediction = predict_next_token("today in new york")
prediction

In [ ]:
tokenizer.sequences_to_texts([prediction])

## Generate New Headlines

In [ ]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([prediction])[0]
        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    return seed_text.title()

In [ ]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']
for seed in seed_texts:
    print(generate_headline(seed, next_words=5))